In [ ]:
import pandas as pd
import tensorflow as tf
import keras
import keras_tuner as kt              


import data_splitter as ds
import data_loader as dl
import feature_extraction as fex
import run_model as rm

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import subprocess

try:
    subprocess.check_output('nvidia-smi')
    print('Nvidia GPU detected!')
except Exception: # this command not being found can raise quite a few different errors depending on the configuration
    print('No Nvidia GPU in system!')

Nvidia GPU detected!


In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

In [ ]:
print("Num CPUs Available: ", len(tf.config.list_physical_devices('CPU')))
print(tf.config.list_physical_devices('CPU'))

In [7]:
KERASTUNER_TUNER_ID="chief"
KERASTUNER_ORACLE_IP="127.0.0.1"
KERASTUNER_ORACLE_PORT="8000"

## Get 25% sample

In [8]:
splitter = ds.DataSplitter(verbose=True)

Total patient_id in training set:  9530
Total patient_id in test set:  2383
Total image_id in training set:  43767
Total image_id in test set:  10939
Total patient_id in training set:  7624
Total patient_id in calibration set:  1906
Total image_id in training set:  35003
Total image_id in calibration set:  8764


In [9]:
# image id of all sample training set after split to calib and train set

train_img_ids = [id for k, v in splitter.trainset.items() for id in v]
print(len(train_img_ids))

35003


In [10]:
print('25% of sample = {} patients, {} images'.format(len(splitter.trainset.keys())*0.25, len(train_img_ids)*0.25))
print('Ratio images : patients = ', len(train_img_ids)/len(splitter.trainset.keys()))

25% of sample = 1906.0 patients, 8750.75 images
Ratio images : patients =  4.591159496327387


In [11]:
metadata = pd.read_csv('train.csv')

In [12]:
# stratified sampling 25% from the training set >> 8,751 images
sample = metadata[metadata['patient_id'].isin(splitter.trainset.keys())]
sample = sample.groupby('cancer', group_keys=False).apply(lambda x: x.sample(frac=0.25))

print(len(sample.patient_id.unique()), len(sample.image_id.unique()))

5530 8751


In [13]:
X = sample.drop(columns='cancer')
y = sample.cancer

X_train, X_validate, y_train, y_validate = train_test_split(X, y, test_size=0.20, random_state=42, stratify=y)
print(len(X_train), len(X_validate))

7000 1751


In [14]:
sample_train_ids = X_train.image_id.unique()
sample_validate_ids = X_validate.image_id.unique()

# Hyperparameter Tuning

## Image Resolution 512 x 512

### Batch size 64 (x 32 cores)

In [15]:
# pre-trained models for feature extraction
IMG_SIZE = (512, 512, 1)

inception = keras.applications.InceptionV3(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),  # should have exactly 3 inputs channels because it is pre-trained on RBG images
    include_top=False
    )

inception.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 255, 255, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 255, 255, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

                                                                                                  
 conv2d_10 (Conv2D)             (None, 61, 61, 96)   82944       ['activation_9[0][0]']           
                                                                                                  
 conv2d_11 (Conv2D)             (None, 61, 61, 32)   6144        ['average_pooling2d[0][0]']      
                                                                                                  
 batch_normalization_5 (BatchNo  (None, 61, 61, 64)  192         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 61, 61, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
          

                                                                  'activation_14[0][0]',          
                                                                  'activation_17[0][0]',          
                                                                  'activation_18[0][0]']          
                                                                                                  
 conv2d_22 (Conv2D)             (None, 61, 61, 64)   18432       ['mixed1[0][0]']                 
                                                                                                  
 batch_normalization_22 (BatchN  (None, 61, 61, 64)  192         ['conv2d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_22 (Activation)     (None, 61, 61, 64)   0           ['batch_normalization_22[0][0]'] 
          

 ormalization)                                                                                    
                                                                                                  
 activation_26 (Activation)     (None, 30, 30, 384)  0           ['batch_normalization_26[0][0]'] 
                                                                                                  
 activation_29 (Activation)     (None, 30, 30, 96)   0           ['batch_normalization_29[0][0]'] 
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 30, 30, 288)  0          ['mixed2[0][0]']                 
                                                                                                  
 mixed3 (Concatenate)           (None, 30, 30, 768)  0           ['activation_26[0][0]',          
                                                                  'activation_29[0][0]',          
          

 activation_39 (Activation)     (None, 30, 30, 192)  0           ['batch_normalization_39[0][0]'] 
                                                                                                  
 mixed4 (Concatenate)           (None, 30, 30, 768)  0           ['activation_30[0][0]',          
                                                                  'activation_33[0][0]',          
                                                                  'activation_38[0][0]',          
                                                                  'activation_39[0][0]']          
                                                                                                  
 conv2d_44 (Conv2D)             (None, 30, 30, 160)  122880      ['mixed4[0][0]']                 
                                                                                                  
 batch_normalization_44 (BatchN  (None, 30, 30, 160)  480        ['conv2d_44[0][0]']              
 ormalizat

                                                                  'activation_49[0][0]']          
                                                                                                  
 conv2d_54 (Conv2D)             (None, 30, 30, 160)  122880      ['mixed5[0][0]']                 
                                                                                                  
 batch_normalization_54 (BatchN  (None, 30, 30, 160)  480        ['conv2d_54[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_54 (Activation)     (None, 30, 30, 160)  0           ['batch_normalization_54[0][0]'] 
                                                                                                  
 conv2d_55 (Conv2D)             (None, 30, 30, 160)  179200      ['activation_54[0][0]']          
          

 ormalization)                                                                                    
                                                                                                  
 activation_64 (Activation)     (None, 30, 30, 192)  0           ['batch_normalization_64[0][0]'] 
                                                                                                  
 conv2d_65 (Conv2D)             (None, 30, 30, 192)  258048      ['activation_64[0][0]']          
                                                                                                  
 batch_normalization_65 (BatchN  (None, 30, 30, 192)  576        ['conv2d_65[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_65 (Activation)     (None, 30, 30, 192)  0           ['batch_normalization_65[0][0]'] 
          

                                                                                                  
 batch_normalization_73 (BatchN  (None, 30, 30, 192)  576        ['conv2d_73[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_73 (Activation)     (None, 30, 30, 192)  0           ['batch_normalization_73[0][0]'] 
                                                                                                  
 conv2d_70 (Conv2D)             (None, 30, 30, 192)  147456      ['mixed7[0][0]']                 
                                                                                                  
 conv2d_74 (Conv2D)             (None, 30, 30, 192)  258048      ['activation_73[0][0]']          
                                                                                                  
 batch_nor

 batch_normalization_83 (BatchN  (None, 14, 14, 384)  1152       ['conv2d_83[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 conv2d_84 (Conv2D)             (None, 14, 14, 192)  245760      ['average_pooling2d_7[0][0]']    
                                                                                                  
 batch_normalization_76 (BatchN  (None, 14, 14, 320)  960        ['conv2d_76[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_78 (Activation)     (None, 14, 14, 384)  0           ['batch_normalization_78[0][0]'] 
                                                                                                  
 activatio

 ormalization)                                                                                    
                                                                                                  
 activation_87 (Activation)     (None, 14, 14, 384)  0           ['batch_normalization_87[0][0]'] 
                                                                                                  
 activation_88 (Activation)     (None, 14, 14, 384)  0           ['batch_normalization_88[0][0]'] 
                                                                                                  
 activation_91 (Activation)     (None, 14, 14, 384)  0           ['batch_normalization_91[0][0]'] 
                                                                                                  
 activation_92 (Activation)     (None, 14, 14, 384)  0           ['batch_normalization_92[0][0]'] 
                                                                                                  
 batch_nor

In [19]:
BATCH_SIZE = 128
IMG_SIZE = (512, 512, 1)
RANDOM_STATE = 42 # the random state that used to create cv splits

BASEPATH = '' # directory storing the preprocessed images of the training set

train_gen, val_gen = rm.get_train_val_generators(
        sample_train_ids,
        sample_validate_ids,
        label_img_dict = splitter.labels,
        patient_img_dict = splitter.trainset,
    
        from_numpy=True,
        basepath=BASEPATH,
        batch_size=BATCH_SIZE,
        img_size=IMG_SIZE,
    
        shuffle=True,
        normalize=(0, 1),
        feature_extractor='CNN',
        CNN_preprocess = keras.applications.inception_v3.preprocess_input
        )

In [20]:
def build_model(hp):
    
    model = keras.Sequential([
        keras.layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
        inception
    ])
    
    # choose an optimal pooling type
    if hp.Choice('globalpooling', ['avg', 'max'])=='max':
        model.add(keras.layers.GlobalMaxPooling2D())
    else:
        model.add(keras.layers.GlobalAveragePooling2D())

    model.add(keras.layers.GlobalAveragePooling2D())    
    model.add(keras.layers.Dense(1, activation='sigmoid'))
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.1, 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss = tf.keras.losses.BinaryCrossentropy(),
        metrics = [keras.metrics.AUC(curve='PR', num_thresholds=100, name='pr_auc')]
    ) 
    
    return model

In [ ]:
# HyperBand algorithm from keras tuner
tuner = kt.Hyperband(
    build_model,
    objective=kt.Objective('val_pr_auc', direction='max'),
    factor=3,
    max_epochs=5,
    hyperband_iterations=1,
    directory='../predicitve_models/keras_tuner',
    project_name='sample_512_batch256x32'
)

tuner.search(
    train_gen,
    #epochs=5,
    callbacks=[tf.keras.callbacks.EarlyStopping('val_pr_auc', patience=2)],
    validation_data=val_gen
    )


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
0.01              |0.01              |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
1                 |1                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
54/54 [==============================] - 8455s 156s/step - loss: 0.2388 - pr_auc: 0.0203 - val_loss: 121612935168.0000 - val_pr_auc: 0.0198
Epoch 2/2
15/54 [=======>......................] - ETA: 1:33:20 - loss: 0.1083 - pr_auc: 0.0196

## Image Resolution 256 x 256

In [22]:
BATCH_SIZE = 8*32 
IMG_SIZE = (256, 256, 1)
RANDOM_STATE = 42 # the random state that used to create cv splits

BASEPATH = '' # directory storing the preprocessed images of the training set

train_gen, val_gen = rm.get_train_val_generators(
        sample_train_ids,
        sample_validate_ids,
        label_img_dict = splitter.labels,
        patient_img_dict = splitter.trainset,
    
        from_numpy=True,
        basepath=BASEPATH,
        batch_size=BATCH_SIZE,
        img_size=IMG_SIZE,
    
        shuffle=True,
        normalize=(0, 1),
        feature_extractor='CNN',
        CNN_preprocess = keras.applications.inception_v3.preprocess_input
        )

In [ ]:
# HyperBand algorithm from keras tuner
tuner = kt.Hyperband(
    build_model,
    objective=kt.Objective('val_pr_auc', direction='max'),
    factor=3,
    max_epochs=5,
    hyperband_iterations=1,
    directory='../predictive_models/keras_tuner',
    project_name='sample_256_batch8x32'
)

tuner.search(
    train_gen,
    #epochs=5,
    callbacks=[tf.keras.callbacks.EarlyStopping('val_pr_auc', patience=2)],
    validation_data=val_gen
    )

Trial 7 Complete [00h 32m 31s]
val_pr_auc: 0.02213541604578495

Best val_pr_auc So Far: 0.024158937856554985
Total elapsed time: 03h 25m 39s

Search: Running Trial #8

Value             |Best Value So Far |Hyperparameter
max               |avg               |globalpooling
0.001             |0.001             |learning_rate
5                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
0                 |1                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/5
 5/27 [====>.........................] - ETA: 6:05 - loss: 0.9048 - pr_auc: 0.0202

In [ ]:
tuner.results_summary()

In [ ]:
print('Best global pooling: ', tuner.get_best_hyperparameters()[0].get('globalpooling'))
print('Best learning rate: ', tuner.get_best_hyperparameters()[0].get('learning_rate'))

### Train model

In [ ]:
EXTRACTOR_PATH = '' # directory where trained feature extractors are stored
BASEPATH = '' # directory of mammogram images

EPOCH = 100 
BATCH_SIZE = 4 
IMG_SIZE = (32, 32, 1)
CORES = 4
RANDOM_STATE = 42 # the random state that used to create cv splits
# LEARNING_RATE = 0.001
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]


CHECKPOINT_PATH = '../predictive_models/inception_checkpoint'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
     filepath = CHECKPOINT_PATH,
     save_weights_only = False,
     monitor = 'val_prc',
     mode = 'max',
     save_best_only = True
     )


history = rm.run_cv_training(
    model_layers = model_layers,
    cv_img = sample_cv_img,

    augment_layers = True,
    base_model = base_model,
    trainable = False,
    loss_func = tf.keras.losses.BinaryCrossentropy(),
    optimizer = keras.optimizers.Adam(),
    metrics = METRICS,

    label_img_dict = sample_label_by_img,
    patient_img_dict = splitter.train,
    basepath = BASEPATH,
    batch_size = BATCH_SIZE, 
    img_size = (IMG_SIZE[0], IMG_SIZE[1]),

    shuffle = True,
    normalize = (-1, 1), 

    feature_extractor_name = 'CNN',
    n_components = None,
    extractor_path = None,
    random_state = RANDOM_STATE,

    checkpoint_path = CHECKPOINT_PATH,
    strategy = None,
    epoch = EPOCH,
    callbacks = None,
    use_multiprocessing = False,
    workers = 1,
    verbose = 1,

    return_none = False
    )

# Train from scratch

## Image resolution 256 x 256

In [14]:
# pre-trained models for feature extraction
IMG_SIZE = (256, 256, 1)

inception = keras.applications.InceptionV3(
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),  # should have exactly 3 inputs channels because it is pre-trained on RBG images
    include_top=False
    )

inception.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 127, 127, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 127, 127, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

 batch_normalization_5 (BatchNo  (None, 29, 29, 64)  192         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_7 (BatchNo  (None, 29, 29, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_10 (BatchN  (None, 29, 29, 96)  288         ['conv2d_10[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_11 (BatchN  (None, 29, 29, 32)  96          ['conv2d_11[0][0]']              
 ormalizat

                                                                                                  
 batch_normalization_22 (BatchN  (None, 29, 29, 64)  192         ['conv2d_22[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_22 (Activation)     (None, 29, 29, 64)   0           ['batch_normalization_22[0][0]'] 
                                                                                                  
 conv2d_20 (Conv2D)             (None, 29, 29, 48)   13824       ['mixed1[0][0]']                 
                                                                                                  
 conv2d_23 (Conv2D)             (None, 29, 29, 96)   55296       ['activation_22[0][0]']          
                                                                                                  
 batch_nor

                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 14, 14, 288)  0          ['mixed2[0][0]']                 
                                                                                                  
 mixed3 (Concatenate)           (None, 14, 14, 768)  0           ['activation_26[0][0]',          
                                                                  'activation_29[0][0]',          
                                                                  'max_pooling2d_2[0][0]']        
                                                                                                  
 conv2d_34 (Conv2D)             (None, 14, 14, 128)  98304       ['mixed3[0][0]']                 
                                                                                                  
 batch_normalization_34 (BatchN  (None, 14, 14, 128)  384        ['conv2d_34[0][0]']              
 ormalizat

                                                                  'activation_39[0][0]']          
                                                                                                  
 conv2d_44 (Conv2D)             (None, 14, 14, 160)  122880      ['mixed4[0][0]']                 
                                                                                                  
 batch_normalization_44 (BatchN  (None, 14, 14, 160)  480        ['conv2d_44[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_44 (Activation)     (None, 14, 14, 160)  0           ['batch_normalization_44[0][0]'] 
                                                                                                  
 conv2d_45 (Conv2D)             (None, 14, 14, 160)  179200      ['activation_44[0][0]']          
          

 ormalization)                                                                                    
                                                                                                  
 activation_54 (Activation)     (None, 14, 14, 160)  0           ['batch_normalization_54[0][0]'] 
                                                                                                  
 conv2d_55 (Conv2D)             (None, 14, 14, 160)  179200      ['activation_54[0][0]']          
                                                                                                  
 batch_normalization_55 (BatchN  (None, 14, 14, 160)  480        ['conv2d_55[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_55 (Activation)     (None, 14, 14, 160)  0           ['batch_normalization_55[0][0]'] 
          

                                                                                                  
 batch_normalization_65 (BatchN  (None, 14, 14, 192)  576        ['conv2d_65[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_65 (Activation)     (None, 14, 14, 192)  0           ['batch_normalization_65[0][0]'] 
                                                                                                  
 conv2d_61 (Conv2D)             (None, 14, 14, 192)  147456      ['mixed6[0][0]']                 
                                                                                                  
 conv2d_66 (Conv2D)             (None, 14, 14, 192)  258048      ['activation_65[0][0]']          
                                                                                                  
 batch_nor

                                                                                                  
 conv2d_70 (Conv2D)             (None, 14, 14, 192)  147456      ['mixed7[0][0]']                 
                                                                                                  
 conv2d_74 (Conv2D)             (None, 14, 14, 192)  258048      ['activation_73[0][0]']          
                                                                                                  
 batch_normalization_70 (BatchN  (None, 14, 14, 192)  576        ['conv2d_70[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_74 (BatchN  (None, 14, 14, 192)  576        ['conv2d_74[0][0]']              
 ormalization)                                                                                    
          

 batch_normalization_76 (BatchN  (None, 6, 6, 320)   960         ['conv2d_76[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_78 (Activation)     (None, 6, 6, 384)    0           ['batch_normalization_78[0][0]'] 
                                                                                                  
 activation_79 (Activation)     (None, 6, 6, 384)    0           ['batch_normalization_79[0][0]'] 
                                                                                                  
 activation_82 (Activation)     (None, 6, 6, 384)    0           ['batch_normalization_82[0][0]'] 
                                                                                                  
 activation_83 (Activation)     (None, 6, 6, 384)    0           ['batch_normalization_83[0][0]'] 
          

                                                                                                  
 activation_91 (Activation)     (None, 6, 6, 384)    0           ['batch_normalization_91[0][0]'] 
                                                                                                  
 activation_92 (Activation)     (None, 6, 6, 384)    0           ['batch_normalization_92[0][0]'] 
                                                                                                  
 batch_normalization_93 (BatchN  (None, 6, 6, 192)   576         ['conv2d_93[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_85 (Activation)     (None, 6, 6, 320)    0           ['batch_normalization_85[0][0]'] 
                                                                                                  
 mixed9_1 

In [16]:
BATCH_SIZE = 8*32 
IMG_SIZE = (256, 256, 1)
RANDOM_STATE = 42 # the random state that used to create cv splits

BASEPATH = '' # directory storing the preprocessed images of the training set

train_gen, val_gen = rm.get_train_val_generators(
        sample_train_ids,
        sample_validate_ids,
        label_img_dict = splitter.labels,
        patient_img_dict = splitter.trainset,
    
        from_numpy=True,
        basepath=BASEPATH,
        batch_size=BATCH_SIZE,
        img_size=IMG_SIZE,
    
        shuffle=True,
        normalize=(0, 1),
        feature_extractor='CNN',
        CNN_preprocess = keras.applications.inception_v3.preprocess_input
        )

In [18]:
def build_model(hp):
    
    model = keras.Sequential([
        keras.layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
        inception
    ])
    
    
    model.add(keras.layers.GlobalAveragePooling2D())   # avg is selected from previous hp search
    model.add(keras.layers.Dense(1, activation='sigmoid')) #****
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.1, 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss = tf.keras.losses.BinaryCrossentropy(),
        metrics = [keras.metrics.AUC(curve='PR', num_thresholds=100, name='pr_auc')]
    ) 
    
    return model

In [19]:
# HyperBand algorithm from keras tuner
tuner = kt.Hyperband(
    build_model,
    objective=kt.Objective('val_pr_auc', direction='max'),
    factor=3,
    max_epochs=5,
    hyperband_iterations=1,
    directory='models/keras_tuner',
    project_name='lr_sample_256_batch8x32_from_scratch'
)

tuner.search(
    train_gen,
    # epochs=5,
    callbacks=[tf.keras.callbacks.EarlyStopping('val_pr_auc', patience=2)],
    validation_data=val_gen
    )

Trial 4 Complete [00h 17m 38s]
val_pr_auc: 0.02083333395421505

Best val_pr_auc So Far: 0.02083333395421505
Total elapsed time: 01h 14m 11s
INFO:tensorflow:Oracle triggered exit


In [20]:
tuner.results_summary()

Results summary
Results in models/keras_tuner/lr_sample_256_batch8x32_from_scratch
Showing 10 best trials
Objective(name="val_pr_auc", direction="max")

Trial 0001 summary
Hyperparameters:
learning_rate: 0.001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.02083333395421505

Trial 0003 summary
Hyperparameters:
learning_rate: 0.1
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.02083333395421505

Trial 0000 summary
Hyperparameters:
learning_rate: 0.01
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.02018229104578495

Trial 0002 summary
Hyperparameters:
learning_rate: 0.0001
tuner/epochs: 2
tuner/initial_epoch: 0
tuner/bracket: 1
tuner/round: 0
Score: 0.01953125


In [21]:
print('Best learning rate: ', tuner.get_best_hyperparameters()[0].get('learning_rate'))

Best learning rate:  0.001


## Image resolution 1024 x 1024

In [15]:
BATCH_SIZE = 8*32 
IMG_SIZE = (1024, 1024, 1)
RANDOM_STATE = 42 # the random state that used to create cv splits

BASEPATH = '' # directory storing the preprocessed images of the training set

train_gen, val_gen = rm.get_train_val_generators(
        sample_train_ids,
        sample_validate_ids,
        label_img_dict = splitter.labels,
        patient_img_dict = splitter.trainset,
    
        from_numpy=True,
        basepath=BASEPATH,
        batch_size=BATCH_SIZE,
        img_size=IMG_SIZE,
    
        shuffle=True,
        normalize=(0, 1),
        feature_extractor='CNN',
        CNN_preprocess = keras.applications.inception_v3.preprocess_input
        )

In [16]:
# pre-trained models for feature extraction
IMG_SIZE = (1024, 1024, 1)

inception = keras.applications.InceptionV3(
    weights='imagenet',  # load weights pre-trained on ImageNet.
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3),  # should have exactly 3 inputs channels because it is pre-trained on RBG images
    include_top=False
    )

inception.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1024, 1024,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, 511, 511, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 511, 511, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                      

                                                                                                  
 average_pooling2d (AveragePool  (None, 125, 125, 19  0          ['max_pooling2d_1[0][0]']        
 ing2D)                         2)                                                                
                                                                                                  
 conv2d_5 (Conv2D)              (None, 125, 125, 64  12288       ['max_pooling2d_1[0][0]']        
                                )                                                                 
                                                                                                  
 conv2d_7 (Conv2D)              (None, 125, 125, 64  76800       ['activation_6[0][0]']           
                                )                                                                 
                                                                                                  
 conv2d_10

                                                                                                  
 conv2d_18 (Conv2D)             (None, 125, 125, 64  16384       ['average_pooling2d_1[0][0]']    
                                )                                                                 
                                                                                                  
 batch_normalization_12 (BatchN  (None, 125, 125, 64  192        ['conv2d_12[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 batch_normalization_14 (BatchN  (None, 125, 125, 64  192        ['conv2d_14[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 batch_nor

                                                                                                  
 batch_normalization_25 (BatchN  (None, 125, 125, 64  192        ['conv2d_25[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 activation_19 (Activation)     (None, 125, 125, 64  0           ['batch_normalization_19[0][0]'] 
                                )                                                                 
                                                                                                  
 activation_21 (Activation)     (None, 125, 125, 64  0           ['batch_normalization_21[0][0]'] 
                                )                                                                 
                                                                                                  
 activatio

 activation_31 (Activation)     (None, 62, 62, 128)  0           ['batch_normalization_31[0][0]'] 
                                                                                                  
 activation_36 (Activation)     (None, 62, 62, 128)  0           ['batch_normalization_36[0][0]'] 
                                                                                                  
 conv2d_32 (Conv2D)             (None, 62, 62, 128)  114688      ['activation_31[0][0]']          
                                                                                                  
 conv2d_37 (Conv2D)             (None, 62, 62, 128)  114688      ['activation_36[0][0]']          
                                                                                                  
 batch_normalization_32 (BatchN  (None, 62, 62, 128)  384        ['conv2d_32[0][0]']              
 ormalization)                                                                                    
          

                                                                                                  
 conv2d_47 (Conv2D)             (None, 62, 62, 160)  179200      ['activation_46[0][0]']          
                                                                                                  
 batch_normalization_42 (BatchN  (None, 62, 62, 160)  480        ['conv2d_42[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_47 (BatchN  (None, 62, 62, 160)  480        ['conv2d_47[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_42 (Activation)     (None, 62, 62, 160)  0           ['batch_normalization_42[0][0]'] 
          

                                                                                                  
 batch_normalization_57 (BatchN  (None, 62, 62, 160)  480        ['conv2d_57[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_52 (Activation)     (None, 62, 62, 160)  0           ['batch_normalization_52[0][0]'] 
                                                                                                  
 activation_57 (Activation)     (None, 62, 62, 160)  0           ['batch_normalization_57[0][0]'] 
                                                                                                  
 average_pooling2d_5 (AveragePo  (None, 62, 62, 768)  0          ['mixed5[0][0]']                 
 oling2D)                                                                                         
          

                                                                                                  
 activation_67 (Activation)     (None, 62, 62, 192)  0           ['batch_normalization_67[0][0]'] 
                                                                                                  
 average_pooling2d_6 (AveragePo  (None, 62, 62, 768)  0          ['mixed6[0][0]']                 
 oling2D)                                                                                         
                                                                                                  
 conv2d_60 (Conv2D)             (None, 62, 62, 192)  147456      ['mixed6[0][0]']                 
                                                                                                  
 conv2d_63 (Conv2D)             (None, 62, 62, 192)  258048      ['activation_62[0][0]']          
                                                                                                  
 conv2d_68

 mixed8 (Concatenate)           (None, 30, 30, 1280  0           ['activation_71[0][0]',          
                                )                                 'activation_75[0][0]',          
                                                                  'max_pooling2d_3[0][0]']        
                                                                                                  
 conv2d_80 (Conv2D)             (None, 30, 30, 448)  573440      ['mixed8[0][0]']                 
                                                                                                  
 batch_normalization_80 (BatchN  (None, 30, 30, 448)  1344       ['conv2d_80[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_80 (Activation)     (None, 30, 30, 448)  0           ['batch_normalization_80[0][0]'] 
          

 batch_normalization_89 (BatchN  (None, 30, 30, 448)  1344       ['conv2d_89[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_89 (Activation)     (None, 30, 30, 448)  0           ['batch_normalization_89[0][0]'] 
                                                                                                  
 conv2d_86 (Conv2D)             (None, 30, 30, 384)  786432      ['mixed9[0][0]']                 
                                                                                                  
 conv2d_90 (Conv2D)             (None, 30, 30, 384)  1548288     ['activation_89[0][0]']          
                                                                                                  
 batch_normalization_86 (BatchN  (None, 30, 30, 384)  1152       ['conv2d_86[0][0]']              
 ormalizat

In [17]:
def build_model(hp):
    
    model = keras.Sequential([
        keras.layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
        inception
    ])
    
    model.add(keras.layers.GlobalAveragePooling2D())   # avg is selected from previous hp search
    model.add(keras.layers.Dense(1, activation='sigmoid')) #****
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.1, 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4])
    
    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss = tf.keras.losses.BinaryCrossentropy(),
        metrics = [keras.metrics.AUC(curve='PR', num_thresholds=100, name='pr_auc')]
    ) 
    
    return model

In [ ]:
# HyperBand algorithm from keras tuner
tuner = kt.Hyperband(
    build_model,
    objective=kt.Objective('val_pr_auc', direction='max'),
    factor=3,
    max_epochs=5,
    #max_trials= 4,
    hyperband_iterations=1,
    directory='../predictive_models/keras_tuner',
    project_name='lr_sample_1024_batch8x32_from_scratch'
)

tuner.search(
    train_gen,
    # epochs=5,
    callbacks=[tf.keras.callbacks.EarlyStopping('val_pr_auc', patience=2)],
    validation_data=val_gen
    )

INFO:tensorflow:Reloading Tuner from models/keras_tuner/lr_sample_1024_batch8x32_from_scratch/tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
0.1               |0.001             |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
1                 |1                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
